In [14]:
from pprint import pprint as p
import sys

p(sys.path)

['',
 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39',
 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32',
 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib',
 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin',
 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']


In [9]:
# p(sys.executable)

In [15]:
sys.path=['', 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip', 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\DLLs', 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\lib', 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39', 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32', 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib', 'C:\\Users\\13474\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin', 'C:\\Users\\13474\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']


In [16]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from fuzzywuzzy import fuzz 

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\13474\Anaconda3\python.exe"
  * The NumPy version is: "1.19.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


In [ ]:
final = pd.read_csv('final_dataset.csv')
final

In [ ]:
df = pd.read_csv('EmployeeEarnings.csv')
df

In [ ]:
pol_only = df[df['DEPARTMENT_NAME'].str.contains('police', case=False, na=False)]
pol_only

In [ ]:
def fixname(name):
    name = name.lower()
    name = name.split(",")
    for i in range(len(name)):
        name[i] = name[i].strip()
    if len(name) == 2:
        return name[1] + ' ' + name[0]
    if len(name) == 3:
        return name[1] + ' ' + name[2] + ' ' + name[0]
    else:
        return name
    
fixname(pol_only['NAME'][57])

In [ ]:
names = pol_only['NAME'].apply(fixname)
pol_only['NAME'] = names
pol_only

In [ ]:
final = pd.read_csv("final_dataset.csv")

In [ ]:
# String similarity between the two fields
def FuzzySimilarity(row):
    address_1 = row['Name']
    address_2 = row['NAME']
    fuzzy_ratio = fuzz.token_set_ratio(address_1,address_2)
    return fuzzy_ratio



In [ ]:
# Create a new column called lastName character that has the first letter of the last name as its separate column

def getLastCh(s):
    "s is a string of the Name/Contributor Columns of the datasets"
    s_list = s.split()
    suffixes = ['jr','jr.','sr','sr.','i','ii','iii']

    # remove suffixes in s_list
    for i in reversed(range(len(s_list))):
        if s_list[i] in suffixes:
            s_list.pop(i)

    lastName = s_list[-1]
    firstCh = lastName[0]
    return firstCh

pol_only['lastNameCh'] = [getLastCh(s) for s in pol_only['NAME']]
final['lastNameCh'] = [getLastCh(s) for s in final['Name']]

In [ ]:
# This program will merge the two dataframes using their lastName characters then apply a string similarity score for each row then we'll filter the string similarity value to create the final dataframe with name matches.

unique_names = list(final['lastNameCh'].unique())

for name in unique_names:
    df2_sub_zip = pol_only[pol_only['lastNameCh'] == name]
    df1_sub_zip = final[final['lastNameCh'] == name]

    df_merge = dd.merge(df1_sub_zip, df2_sub_zip, how='outer', left_on='lastNameCh', right_on='lastNameCh')
    
    df_merge['Fuzzy Similarity'] = df_merge.apply(lambda row: FuzzySimilarity(row), axis=1)
    
    # You can adjust this number for a more selective fuzzy similarity merge
    Fuzzy_Filter = df_merge[df_merge['Fuzzy Similarity'] > 85]
    
    title = "./fuzzyDatasets/merge_df_name_" + name + ".csv"
    Fuzzy_Filter.to_csv(title, encoding = "utf-8")

In [ ]:
# This will create the list

list_of_csv_titles = []

for name in unique_names:
    title = "./fuzzyDatasets/merge_df_name_" + name + ".csv"
    list_of_csv_titles.append(title)
    
print(list_of_csv_titles)

In [ ]:
# Merging all the batches

df_merge_final = pd.DataFrame()
for files in list_of_csv_titles:
    data = pd.read_csv(files)
    df_merge_final = df_merge_final.append(data)
df_merge_final

In [ ]:
df_merge_final

In [ ]:
df_merge_final.to_csv("with_earnings.csv")